In [1]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

#### load data

In [2]:
df_train = pd.read_csv("../data/processed/ultimate_train.csv")
df_val = pd.read_csv("../data/processed/ultimate_val.csv")
df_test = pd.read_csv("../data/processed/ultimate_test_us.csv")

In [3]:
df_train = df_train.fillna(method='ffill')
df_val = df_val.fillna(method='ffill')
df_test = df_test.fillna(method='ffill')

In [4]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(47430, 105)
(11858, 105)
(14823, 105)


#### train-test split

In [5]:
y_train = df_train.log_price
X_train = df_train.drop('log_price', axis=1)

y_val = df_val.log_price
X_val = df_val.drop('log_price', axis=1)

y_test = df_test.log_price
X_test = df_test.drop('log_price', axis=1)

y_train_final = pd.concat([y_train, y_val], axis=0)
X_train_final = pd.concat([X_train, X_val], axis=0)

y_val_exp = np.exp(y_val)
y_test_exp = np.exp(y_test)

In [6]:
X_train_final.shape

(59288, 104)

### Neural Network

**response: original price**

#### default model:

In [7]:
# defualt hyperparameters
default_mlp = MLPRegressor().fit(X_train, y_train)

In [8]:
print(median_absolute_error(y_val_exp, np.exp(default_mlp.predict(X_val))))
print(mean_absolute_error(y_val_exp, np.exp(default_mlp.predict(X_val))))

43.698565070930535
80.78545837770523


#### Random Search

In [9]:
mlp = MLPRegressor(max_iter=200, early_stopping=True, n_iter_no_change=5)
# parameter
layers = [(200,), (220,), (300,), (200, 200), (120, 120)]

param_dist_mlp = dict(hidden_layer_sizes=layers, 
                      learning_rate=['constant','adaptive'],
                      learning_rate_init = [0.001, 0.0001, 0.01],
                      alpha=[0.0001, 0.002 ,0.05])

# randomized search
rand_mlp = RandomizedSearchCV(mlp, param_dist_mlp, random_state=0, 
                              n_iter=10, cv=3, verbose=0, n_jobs=-1)

In [20]:
best_model_mlp = rand_mlp.fit(X_train, y_train)

In [21]:
print('Best hidden_layer_sizes:', best_model_mlp.best_estimator_.get_params()['hidden_layer_sizes'])
print('Best learning_rate:', best_model_mlp.best_estimator_.get_params()['learning_rate'])
print('Best alpha:', best_model_mlp.best_estimator_.get_params()['alpha'])
print('Best learning_rate_init:', best_model_mlp.best_estimator_.get_params()['learning_rate_init'])

Best hidden_layer_sizes: (200, 200)
Best learning_rate: adaptive
Best alpha: 0.002
Best learning_rate_init: 0.01


In [22]:
best_mlp_exp = MLPRegressor(hidden_layer_sizes=(200, 200), 
                         learning_rate='adaptive', learning_rate_init=0.01, alpha=0.002, 
                         max_iter=200, early_stopping=True, 
                         n_iter_no_change=5).fit(X_train, y_train)

In [23]:
print(median_absolute_error(y_val_exp, np.exp(best_mlp_exp.predict(X_val))))
print(mean_absolute_error(y_val_exp, np.exp(best_mlp_exp.predict(X_val))))

30.93008569635821
61.26414189451064


### Training neural net using the best model

In [26]:
final_mlp = MLPRegressor(hidden_layer_sizes=(200, 200), 
                         learning_rate='adaptive', learning_rate_init=0.0001, alpha=0.02, 
                         max_iter=200, early_stopping=True, 
                         n_iter_no_change=5).fit(X_train_final, y_train_final)

In [27]:
print(median_absolute_error(y_test_exp, np.exp(final_mlp.predict(X_test))))
print(mean_absolute_error(y_test_exp, np.exp(final_mlp.predict(X_test))))

36.55989785399066
74.95400539113488


**defualt model**

In [28]:
default_final_mlp = MLPRegressor().fit(X_train_final, y_train_final)

In [29]:
print(median_absolute_error(y_test_exp, np.exp(default_final_mlp.predict(X_test))))
print(mean_absolute_error(y_test_exp, np.exp(default_final_mlp.predict(X_test))))

36.04768793734371
69.88167184133258
